In [1]:
# Loading the Packages
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')
import os
from pathlib import Path
import pickle
from tqdm import tqdm

import numpy as np
import pandas as pd
import scanpy as sc

from tifffile import imread, imwrite
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams.update({
    "pgf.texsystem": "xelatex",      # 使用 XeLaTeX，如果不需要 LaTeX 公式渲染，可以省略
    'font.family': 'serif',          # 字体设置为衬线字体
    'text.usetex': False,            # 禁用 LaTeX，使用 Matplotlib 内置文字渲染
    'pgf.rcfonts': False,            # 禁用 pgf 的默认字体管理
    'pdf.fonttype': 42,              # 确保字体为 TrueType 格式，可被 Illustrator 编辑
    'ps.fonttype': 42,               # EPS 文件也使用 TrueType 格式
    'figure.dpi': 300,               # 设置图形分辨率
    'savefig.dpi': 300,              # 保存的图形文件分辨率
    'axes.unicode_minus': False,     # 避免负号问题
})

In [2]:
# workdir 
BASE_DIR = Path(r'G:\spatial_data')
RUN_ID = '20230523_HCC_PRISM_probe_refined'
src_path = BASE_DIR / 'processed' / f'{RUN_ID}'
analysis_path = BASE_DIR / 'analysis' / f'{RUN_ID}'

# Load one slide exp
segmend_path = src_path / "segmented"
typ_path = analysis_path / "cell_typing"
projection_path = analysis_path / "projection"
os.makedirs(projection_path, exist_ok=True)

In [3]:
def show_cluster(hulls, type_indices, cluster_list, COI='', cluster_colormap=["red"] * 200, ax='', linewidth=0.1, name='projection', show=True, save=False, outpath='',showname=True, verbose=True):
    cell = 0
    for ind in cluster_list:
        for idx in tqdm(type_indices[ind], desc=f'cell for cluster_{ind}', disable=not verbose):
            idx = int(idx)
            try:
                ax.fill(hulls[idx][:, 1], hulls[idx][:, 0],color=cluster_colormap[ind], linewidth=linewidth, alpha=1)
                if cluster_colormap[ind] != (0.9,0.9,0.9):
                    cell += 1
            except KeyError: pass
    ax.set_xlim([0, 37500])
    ax.set_ylim([0, 35000])
    if showname: ax.set_title(f"{COI}, cell_num={cell}, linewidth={linewidth}")
    else: ax.set_title(f"cell_num={cell}")
    if verbose: print(f'{cell} cells have been ploted.')
    
    if save: plt.savefig(outpath)
    elif show: plt.show()
    else: pass

# create hulls

In [ ]:
# cell show
import pandas as pd
import numpy as np
from scipy.spatial import ConvexHull
import matplotlib.pyplot as plt

def create_hull(rna_pos):
    hulls = {}
    df_group = rna_pos.groupby("Cell Index")
    for group in tqdm(df_group, desc="hull"):
        coordinates = group[1][["Y", "X"]].values
        try: hull = ConvexHull(coordinates)
        except: continue
        coordinate_path = np.vstack((coordinates[hull.vertices, 0], coordinates[hull.vertices, 1])).T
        hulls[group[0]] = coordinate_path
    return hulls

In [ ]:
rna_pos = pd.read_csv(segmend_path / 'rna_labeled.csv')
hulls = create_hull(rna_pos)
pickle.dump(hulls, open(projection_path / 'hulls.pkl', 'wb'))

# cell type

In [4]:
adata = sc.read_h5ad(os.path.join(typ_path, 'adata.h5ad'))
adata

AnnData object with n_obs × n_vars = 80396 × 31
    obs: 'dataset', 'n_genes', 'n_counts', 'leiden_type', 'type', 'leiden_subtype', 'subtype'
    var: 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'mean', 'std'
    uns: 'leiden', 'log1p', 'neighbors', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap', 'spatial'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [5]:
with open(projection_path / 'hulls_type.pkl', 'rb') as f: hulls = pickle.load(f)

In [6]:
import yaml
cell_typing_params = yaml.safe_load(open(os.path.join(analysis_path, 'cell_typing_params.yaml'), 'r'))
type_colormap = cell_typing_params['type_colormap']
subtype_colormap = dict()

leiden_annotation = cell_typing_params['leiden_annotation']
leiden_subtype_annotation = {}
for type_key, subtypes in leiden_annotation.items():
    type_values = []
    type_color = type_colormap[type_key]
    for subtype_key, values in subtypes.items():
        type_values.extend(values)
        leiden_subtype_annotation[subtype_key] = values
        subtype_colormap[subtype_key] = type_color
    leiden_annotation[type_key] = type_values

## plot differnent liver

In [ ]:
UMAP_leiden_plot(adata=adata[~adata.obs['type'].isin(['other'])], DOI=['PRISM_HCC'], datatype='harmony', color='type', FOI='HCC', palette=type_colormap, legend_loc='right margin', 
                 show=False, save=True, out_path=os.path.join(figure_path, 'cell_type_UMAP.png'))

In [ ]:
liver_cells = adata[adata.obs['type']=='Liver']
print(liver_cells.obs.leiden.unique())
liver_cells

In [ ]:
cancer_liver_leiden = [str(_) for _ in [11,20,32,90,77,70,65,74,42]]
cancer_liver = liver_cells[liver_cells.obs.leiden.isin(cancer_liver_leiden)]
other_liver_leiden = [str(_) for _ in [40,62,7,73,92,57]]
other_liver = liver_cells[liver_cells.obs.leiden.isin(other_liver_leiden)]

liver_cells.obs['liver_subtype'] = ['-1']*len(liver_cells)
liver_cells.obs.loc[cancer_liver.obs.index, 'liver_subtype'] = ['cancer_liver']*len(cancer_liver)
liver_cells.obs.loc[other_liver.obs.index, 'liver_subtype'] = ['other_liver']*len(other_liver)

In [ ]:
tmp_adata = liver_cells.copy()
tmp_adata.obs.index = [_.split('-')[0] for _ in tmp_adata.obs.index]
hulls, type_indices = create_hull(tmp_adata, clus_obs="liver_subtype", cont_thre=5, 
                                rna_pos = pd.read_csv(os.path.join(base_path, 'segmented', "rna_labeled_0818_HCC.csv")))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
show_cluster(hulls, type_indices, cluster_list=['cancer_liver', 'other_liver'], 
             cluster_colormap={"cancer_liver":(1,0,0), "other_liver":(0,0,1)}, 
            linewidth=2, ax=ax, show=False, save=False)
cell_num_ = len(tmp_adata)
fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'projection_dif_type_Liver.png'), dpi=300)
plt.close()

## type

In [ ]:
import pickle

# plot type cluster if not exist
if not os.path.exists(os.path.join(typ_path, 'hulls_type.pkl')) or not os.path.exists(os.path.join(typ_path, 'type_indices.pkl')):
    # plot type cluster
    tmp_adata = adata[adata.obs.type != 'other']
    tmp_adata.obs.index = [_.split('-')[0] for _ in tmp_adata.obs.index]
    hulls, type_indices = create_hull(tmp_adata, clus_obs="type", cont_thre=5, 
                                    rna_pos = pd.read_csv(os.path.join(base_path, 'segmented', "rna_labeled_0818_HCC.csv")))
    # save hulls and type_indices
    with open(os.path.join(typ_path, 'hulls_type.pkl'), 'wb') as f: pickle.dump(hulls, f)
    with open(os.path.join(typ_path, 'type_indices.pkl'), 'wb') as f: pickle.dump(type_indices, f)

else:
    with open(os.path.join(typ_path, 'hulls_type.pkl'), 'rb') as f: hulls = pickle.load(f)
    with open(os.path.join(typ_path, 'type_indices.pkl'), 'rb') as f: type_indices = pickle.load(f)

### all in one

In [10]:
from skimage.draw import polygon
from PIL import Image
from copy import deepcopy

downsize = 0.25
# 参数设置
pixel_resolution = 1  # 调整此值以提高分辨率（例如10.0）
background_color = (255, 255, 255)  # 背景色（白色）

adata_tmp = adata.copy()
adata_tmp.obs['X_pos'] = adata_tmp.obsm['spatial'][:, 1] * downsize
adata_tmp.obs['Y_pos'] = adata_tmp.obsm['spatial'][:, 0] * downsize

# 获取坐标范围
xmin, xmax = adata_tmp.obs['X_pos'].min(), adata_tmp.obs['X_pos'].max()
ymin, ymax = adata_tmp.obs['Y_pos'].min(), adata_tmp.obs['Y_pos'].max()

# 计算画布尺寸（增加边界避免坐标溢出）
width_px = int((xmax - xmin) * pixel_resolution) + 200
height_px = int((ymax - ymin) * pixel_resolution) + 200

# 创建RGB画布
canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)

# 坐标转换函数（确保x,y顺序正确）
def coord_to_px(x, y):
    px = int((x - xmin) * pixel_resolution) + 100
    py = int((y - ymin) * pixel_resolution) + 100
    return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)

# 遍历细胞并绘制
for idx, row in adata_tmp.obs.iterrows():
    idx = int(idx)
    try:
        if idx not in hulls:
            print(f"Warning: {idx} not in hulls, skipping.")
            continue
        hull = hulls[idx] * downsize
        celltype = row['type']
        color = type_colormap[celltype]
        color = tuple(int(255 * x) for x in color)
        if isinstance(color, str):
            if color.startswith('#'):
                color = color.lstrip('#')
                color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
            else:
                # 处理命名颜色（如'red'需映射到RGB）
                from matplotlib.colors import to_rgb
                color = tuple(int(255 * x) for x in to_rgb(color))
        # 如果hull格式为[y, x]，需转换为[x, y]
        hull_xy = hull[:, [1, 0]]  # 假设hull原始格式为[y, x]x
        # 转换坐标到像素空间
        coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
        # 生成多边形像素坐标
        rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
        canvas[rr, cc, 0] = color[0]  # 红
        canvas[rr, cc, 1] = color[1]  # 绿
        canvas[rr, cc, 2] = color[2]  # 蓝
    except KeyError: continue

imwrite(str(projection_path / f'cell_type.tiff'), np.flipud(canvas), photometric='rgb')
# img = Image.fromarray(canvas)
# img.save(str(projection_path / f"DC_tumor.png"), format='PNG')

In [ ]:
# tmp_adata = adata[adata.obs.type!='other']
# linewidth = 0.1
# fig, ax = plt.subplots(figsize=(10, 10))
# show_cluster(hulls, type_indices, cluster_list=list(leiden_type_dict.keys()), cluster_colormap=type_colormap, 
#               linewidth=linewidth, ax=ax, show=False, save=False, verbose=False)
# cell_num_ = len(tmp_adata)
# fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
# plt.tight_layout()
# plt.savefig(os.path.join(figure_path, f'projection_type_linewidth={linewidth}.pdf'))
# plt.close()

In [ ]:
# from IPython.display import clear_output

# cluster_all = [_ for _ in leiden_type_dict.keys()]
# cluster_of_interest = cluster_all
# dpi = 100
# linewidth = 1.5
# projection_path = os.path.join(figure_path, 'projection_type_brown')
# if not os.path.exists(projection_path): os.makedirs(projection_path)
# for cluster_num, cluster_type in enumerate(cluster_of_interest):
#     clear_output()
#     print('{:.1f}%: {}'.format((cluster_num+1)/len(cluster_of_interest)*100, cluster_type))
#     tmp_colormap = {_:(0.9,0.9,0.9) for _ in cluster_all}
#     # tmp_colormap[cluster_type] = type_colormap[cluster_type]
#     tmp_colormap[cluster_type] = 'brown'
#     tmp_TOI = [_ for _ in cluster_all]
#     tmp_TOI.remove(cluster_type)
#     tmp_TOI = tmp_TOI + [cluster_type]
#     fig, ax = plt.subplots(figsize=(10, 10))
#     show_cluster(hulls, type_indices, cluster_list = tmp_TOI, cluster_colormap=tmp_colormap, COI=cluster_type,
#                 ax=ax, show=False, save=False, linewidth=linewidth, name=cluster_type, verbose=False)
#     fig.suptitle(f'2D_Projection, cell_num={len(tmp_adata)}\n\n', fontsize=20)
#     plt.tight_layout()
#     plt.savefig(os.path.join(projection_path, rf'{cluster_num}_{cluster_type}_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches='tight')
#     plt.close()

### non-immune

#### projection all in one

In [ ]:
tmp_adata = adata[adata.obs.type!='other']
tmp_adata[tmp_adata.obs.type.isin(['Tumor', 'CAF', 'Liver', 'Endo', 'Ep'])]

In [ ]:
non_immune_colormap = dict()
for type_name in type_colormap.keys():
  if type_name in ['Tumor', 'CAF', 'Liver', 'Endo', 'Ep']: non_immune_colormap[type_name]=type_colormap[type_name]
  else: non_immune_colormap[type_name]=(0.9,0.9,0.9)  
non_immune_colormap

In [ ]:
non_immune_colormap = dict()
for type_name in type_colormap.keys():
  if type_name in ['Tumor', 'CAF', 'Liver', 'Endo', 'Ep']: non_immune_colormap[type_name]=type_colormap[type_name]
  else: non_immune_colormap[type_name]=(0.9,0.9,0.9)  

linewidth = 1.5
dpi = 100
tmp_adata = adata[adata.obs.type!='other']
fig, ax = plt.subplots(figsize=(10, 10))
show_cluster(hulls, type_indices, cluster_list=list(leiden_type_dict.keys())[5:]+['Tumor', 'CAF', 'Endo', 'Ep', 'Liver'], cluster_colormap=non_immune_colormap, 
             linewidth=linewidth, ax=ax, show=False, save=False, verbose=False)
cell_num_ = len(tmp_adata[tmp_adata.obs.type.isin(['Tumor', 'CAF', 'Liver', 'Endo', 'Ep'])])
fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'projection_type_non-immune_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi)
plt.close()

#### Type separate

In [ ]:
linewidth = 1.5
dpi = 100
projection_path = os.path.join(figure_path, 'projection_type_color')
if not os.path.exists(projection_path): os.makedirs(projection_path)
for type_of_i in ['Tumor', 'CAF', 'Endo', 'Ep', 'Liver',]:
    non_immune_colormap = dict()
    for type_name in type_colormap.keys():
        if type_name in [type_of_i]: non_immune_colormap[type_name]=type_colormap[type_name]
        else: non_immune_colormap[type_name]=(0.9,0.9,0.9)  
    tmp_list = ['Endo', 'Ep', 'Liver', 'Tumor', 'CAF', ]
    tmp_list.remove(type_of_i)
    fig, ax = plt.subplots(figsize=(10, 10))
    show_cluster(hulls, type_indices, cluster_list=list(leiden_type_dict.keys())[5:]+tmp_list+[type_of_i], 
                cluster_colormap=non_immune_colormap, linewidth=linewidth, ax=ax, show=False, save=False, name=type_of_i, verbose=False)

    cell_num_ = len(tmp_adata[tmp_adata.obs.type.isin(['Tumor', 'CAF', 'Liver', 'Endo', 'Ep'])])
    fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
    plt.savefig(os.path.join(projection_path, f'projection_type_non-immune_{type_of_i}_width={linewidth}_dpi={dpi}.png'), dpi=dpi)
    plt.close()

### immune

#### projection all in one

In [ ]:
immune_colormap = dict()
for type_name in type_colormap.keys():
  if type_name not in ['Tumor', 'CAF', 'Liver', 'Endo', 'Ep']: immune_colormap[type_name]=type_colormap[type_name]
  else: immune_colormap[type_name]=(0.9,0.9,0.9)

linewidth = 1.5
dpi = 100
projection_path = os.path.join(figure_path, 'projection_type_color')
if not os.path.exists(projection_path): os.makedirs(projection_path)
fig, ax = plt.subplots(figsize=(10, 10))
show_cluster(hulls, type_indices, cluster_list=list(leiden_type_dict.keys()), 
             cluster_colormap=immune_colormap, linewidth=linewidth, ax=ax, show=False, save=False, name=cluster_type)
cell_num_ = len(tmp_adata[~tmp_adata.obs.type.isin(['Tumor', 'CAF', 'Liver', 'Endo', 'Ep'])])
fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
plt.savefig(os.path.join(projection_path, f'projection_type_immune_width={linewidth}_dpi={dpi}.pdf'))
plt.close()

#### Type Of Interest

In [ ]:
from IPython.display import clear_output

cluster_all = [_ for _ in leiden_type_dict.keys()]
cluster_of_interest = ['Neutrophil', 'Macrophage']
ncols = int(-(-len(cluster_of_interest)**(1/2)//1))
nrows = -(-len(cluster_of_interest)//ncols)
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))
# for cluster_num, cluster_type in enumerate(cluster_of_interest):
for cluster_num, cluster_type in enumerate(cluster_of_interest):
    clear_output()
    print('{:.1f}%: {}'.format((cluster_num+1)/len(cluster_of_interest)*100, cluster_type))
    tmp_colormap = {_:(0.9,0.9,0.9) for _ in cluster_all}
    tmp_colormap[cluster_type] = type_colormap[cluster_type]
    tmp_TOI = [_ for _ in cluster_all]
    tmp_TOI.remove(cluster_type)
    tmp_TOI = tmp_TOI + [cluster_type]
    ax_tmp = ax[cluster_num]
    # ax_tmp = ax[cluster_num // ncols][cluster_num % ncols],
    show_cluster(hulls, type_indices, cluster_list=tmp_TOI, cluster_colormap=tmp_colormap, COI=cluster_type, 
                 ax=ax_tmp, show=False, save=False, linewidth=1.5, name=cluster_type)
    ax_tmp.set_xlabel("")
    ax_tmp.set_ylabel("")
    ax_tmp.set_xticks([])
    ax_tmp.set_yticks([])
fig.suptitle(f'2D_Projection, cell_num={len(tmp_adata)}\n\n', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(figure_path,'projection_type_Neutro-Macro.png'), bbox_inches = 'tight')
plt.savefig(os.path.join(figure_path,'projection_type_Neutro-Macro.pdf'), bbox_inches = 'tight')
plt.close()

In [ ]:
# fig, ax = plt.subplots(figsize=(10, 10))
# show_cluster(hulls, type_indices, cluster_list = list(leiden_type_dict.keys()), cluster_colormap = type_colormap, 
#               linewidth=0.1, ax=ax, show=False, save=False, name=cluster_type)
# fig.suptitle(f'2D_Projection, cell_num={len(tmp_adata)}\n\n', fontsize=20)
# plt.show()

In [ ]:
projection_path = os.path.join(figure_path, 'projection_type_color')
if not os.path.exists(projection_path): os.makedirs(projection_path)
immune_interest = ['Monocyte', 'NK', 'CD8+', 'Neutrophil', 'Mast']
linewidth = 1.5
dpi = 100
for immune_type in immune_interest:
  immune_colormap = dict()
  for type_name in type_colormap.keys():
    if type_name != immune_type: immune_colormap[type_name]=(0.9,0.9,0.9)  
    else: immune_colormap[type_name]=type_colormap[type_name]

  plot_type = list(leiden_type_dict.keys())
  plot_type.remove(immune_type)
  plot_type +=[immune_type]

  fig, ax = plt.subplots(figsize=(10, 10))
  show_cluster(hulls, type_indices, cluster_list=plot_type, 
                cluster_colormap = immune_colormap, linewidth=linewidth, ax=ax, show=False, save=False, name=immune_type, verbose=False)
  fig.suptitle(f'2D_Projection\n\n', fontsize=20)
  plt.savefig(os.path.join(projection_path, f'type_immune_{immune_type}_spatial_projection_merged_width={linewidth}_dpi={dpi}.png'), dpi=dpi)
  plt.close()

### DC and Tumor

In [ ]:
from copy import deepcopy
type_colormap_tmp = deepcopy(type_colormap)
for key in type_colormap_tmp.keys():
    if key not in ['Tumor', 'DC']: type_colormap_tmp[key] = (0.9,0.9,0.9)
    else: type_colormap_tmp[key] = type_colormap[key]

In [ ]:
type_colormap_tmp

In [ ]:
from skimage.draw import polygon
from PIL import Image

# 参数设置
pixel_resolution = 1  # 调整此值以提高分辨率（例如10.0）
background_color = (255, 255, 255)  # 背景色（白色）

adata_tmp = adata.copy()
adata_tmp.obs['X_pos'] = adata_tmp.obsm['spatial'][:, 1]
adata_tmp.obs['Y_pos'] = adata_tmp.obsm['spatial'][:, 0]

# 获取坐标范围
xmin, xmax = adata_tmp.obs['X_pos'].min(), adata_tmp.obs['X_pos'].max()
ymin, ymax = adata_tmp.obs['Y_pos'].min(), adata_tmp.obs['Y_pos'].max()

# 计算画布尺寸（增加边界避免坐标溢出）
width_px = int((xmax - xmin) * pixel_resolution) + 200
height_px = int((ymax - ymin) * pixel_resolution) + 200

# 创建RGB画布
canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)

# 坐标转换函数（确保x,y顺序正确）
def coord_to_px(x, y):
    px = int((x - xmin) * pixel_resolution) + 100
    py = int((y - ymin) * pixel_resolution) + 100
    return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)

# 遍历细胞并绘制
for idx, row in adata_tmp.obs.iterrows():
    idx = int(idx)
    try:
        if idx not in hulls:
            print(f"Warning: {idx} not in hulls, skipping.")
            continue
        hull = hulls[idx]
        celltype = row['type']
        color = type_colormap_tmp[celltype]
        color = tuple(int(255 * x) for x in color)
        if isinstance(color, str):
            if color.startswith('#'):
                color = color.lstrip('#')
                color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
            else:
                # 处理命名颜色（如'red'需映射到RGB）
                from matplotlib.colors import to_rgb
                color = tuple(int(255 * x) for x in to_rgb(color))
        # 如果hull格式为[y, x]，需转换为[x, y]
        hull_xy = hull[:, [1, 0]]  # 假设hull原始格式为[y, x]
        # 转换坐标到像素空间
        coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
        # 生成多边形像素坐标
        rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
        canvas[rr, cc, 0] = color[0]  # 红
        canvas[rr, cc, 1] = color[1]  # 绿
        canvas[rr, cc, 2] = color[2]  # 蓝
    except KeyError: continue

imwrite(str(projection_path / 'DC_tumor.tiff'), np.flipud(canvas), photometric='rgb')
# img = Image.fromarray(canvas)
# img.save(str(projection_path / f"DC_tumor.png"), format='PNG')

## subtype

In [ ]:
adata = sc.read_h5ad(os.path.join(typ_path, 'adata.h5ad'))

### subtype all in one

In [ ]:
from IPython.display import clear_output

cluster_of_interest = list(leiden_subtype_dict.keys())
ncols = int(-(-len(cluster_of_interest)**(1/2)//1))
nrows = -(-len(cluster_of_interest)//ncols)
linewidth = 1.5
dpi = 100
fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))
for cluster_num, cluster_type in enumerate(cluster_of_interest):
    clear_output()
    ax_tmp = ax[cluster_num // ncols][cluster_num % ncols]
    ax_tmp.set_xlabel("")
    ax_tmp.set_ylabel("")
    ax_tmp.set_xticks([])
    ax_tmp.set_yticks([])
    print('{:.1f}%: {}'.format((cluster_num+1)/len(cluster_of_interest)*100, cluster_type))
    tmp_colormap = {_:(0.9,0.9,0.9) for _ in cluster_of_interest}
    tmp_colormap[cluster_type] = subtype_colormap[cluster_type]
    tmp_TOI = [_ for _ in cluster_of_interest]
    tmp_TOI.remove(cluster_type)
    tmp_TOI = tmp_TOI + [cluster_type]
    show_cluster(hulls, type_indices, cluster_list=tmp_TOI, COI=cluster_type,
                 cluster_colormap=tmp_colormap, linewidth=linewidth, ax=ax_tmp, show=False, save=False, name=cluster_type, verbose=False) 

fig.suptitle(f'2D_Projection, cell_num={len(tmp_adata)}\n\n', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(figure_path, f'projection_subtype_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches = 'tight')
plt.close()

### subtype separate

In [ ]:
from IPython.display import clear_output

projection_path = os.path.join(figure_path, 'projection_subtype_brown')
if not os.path.exists(projection_path): os.makedirs(projection_path)
cluster_all = [_ for _ in leiden_subtype_dict.keys()]
cluster_of_interest = cluster_all
linewidth = 1.5
dpi = 100
for cluster_num, cluster_type in enumerate(cluster_of_interest):
    clear_output()
    print('{:.1f}%: {}'.format((cluster_num+1)/len(cluster_of_interest)*100, cluster_type))
    tmp_colormap = {_:(0.9,0.9,0.9) for _ in cluster_all}
    tmp_colormap[cluster_type] = 'brown'
    tmp_TOI = [_ for _ in cluster_all]
    tmp_TOI.remove(cluster_type)
    tmp_TOI = tmp_TOI + [cluster_type]
    fig, ax = plt.subplots(figsize=(10, 10))
    show_cluster(hulls, type_indices, cluster_list=tmp_TOI, 
                cluster_colormap=tmp_colormap, COI=cluster_type, linewidth=linewidth, ax=ax, show=False, save=False, name=cluster_type, verbose=False)
    fig.suptitle(f'2D_Projection, cell_num={len(tmp_adata)}\n\n', fontsize=20)
    plt.savefig(os.path.join(projection_path, rf'{cluster_num}_{cluster_type}_linewidth_{linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches='tight')
    plt.close()

### Tumor

In [ ]:
# type_colormap = {
#     'Liver':(1,0.392,0), 'Tumor':(0.751,0.491,0), 'Endo':(1,0,1), 'Ep':(0,1,0), 'CAF':(0,0,1),
#     'DC':(1,0.259,0), 'Mait':(1,0,0.434), 'Mast':(1,0,0), 
#     'Monocyte':(0,0.471,1), 'Neutrophil':(0.6,0.6,0), 'Macrophage':(0.2,0.6,0),
#     'CD4+':(0.5,0.5,0.5), 'CD8+':(1,0.8,0), 'T_reg':(0,1,0.672), 
#     'proliferation':(0,1,0.636), 'B':(0,1,1), 'NK':(1,0,0), 'other':(0.9,0.9,0.9)}

In [ ]:
tmp_colormap = dict()
for type_name in type_colormap.keys(): tmp_colormap[type_name]=(0.9,0.9,0.9)
tmp_colormap['Tumor'] = type_colormap['Tumor']

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
sc.pl.umap(adata[adata.obs.subtype!='other'], 
           color='type', palette=tmp_colormap, legend_fontsize=7, ax=ax, show=False)
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'type_nonimmune_tumor_UMAP.png'), bbox_inches = 'tight')
plt.close()


Subtype_of_interest = ['Tumor_proliferation', 'Tumor_AFP+', 'Tumor_GPC3+']
tumor_colormap = dict()
for type_name in subtype_colormap.keys(): tumor_colormap[type_name]=(0.9,0.9,0.9)  

tumor_colormap['Tumor_AFP+']='#FF8B40'
tumor_colormap['Tumor_GPC3+']="#A468D5"
tumor_colormap['Tumor_proliferation']='#5CCDC9'

linewidth = 1.5
dpi = 100
plot_type = list(leiden_subtype_dict.keys())
for _ in Subtype_of_interest:
  plot_type.remove(_)
plot_type += Subtype_of_interest

fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
show_cluster(hulls, type_indices, cluster_list=plot_type, cluster_colormap=tumor_colormap, 
              linewidth=1.5, ax=ax[0], show=False, save=False, name=cluster_type, showname=False, verbose=False)
sc.pl.umap(
  adata[adata.obs.subtype!='other'], color='subtype', 
  palette=tumor_colormap, legend_fontsize=7, ax=ax[1], show=False)

fig.suptitle(f'2D_Projection_and_UMAP\n\n', fontsize=20)
plt.savefig(os.path.join(figure_path, f'subtype_non-immune_tumor_projection_and_UMAP_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches='tight')
plt.close()

### Monocyte

In [ ]:
tmp_colormap = dict()
for type_name in type_colormap.keys(): tmp_colormap[type_name]=(0.9,0.9,0.9)
tmp_colormap['Monocyte'] = type_colormap['Monocyte']

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
sc.pl.umap(adata[adata.obs.subtype!='other'], 
           color='type', palette=tmp_colormap, legend_fontsize=7, ax=ax, show=False)
plt.savefig(os.path.join(figure_path, 'type_nonimmune_Monocyte_UMAP.png'), bbox_inches='tight')
plt.close()

In [ ]:
Subtype_of_interest = ['Monocyte_CD14+', 'Monocyte_CD14+, CD16+', 'Monocyte_CD16+']
monocyte_colormap = dict()
for type_name in subtype_colormap.keys():
  monocyte_colormap[type_name]=(0.9,0.9,0.9)  
monocyte_colormap['Monocyte_CD14+']=(0.9,0,1)
monocyte_colormap['Monocyte_CD14+, CD16+']=(0,0,1)
monocyte_colormap['Monocyte_CD16+']=(0,0.9,1)

plot_type = list(leiden_subtype_dict.keys())
for _ in Subtype_of_interest:
  plot_type.remove(_)
plot_type += Subtype_of_interest

linewidth = 1.5
dpi = 100
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
show_cluster(hulls, type_indices, cluster_list=plot_type, 
              cluster_colormap = monocyte_colormap, linewidth=linewidth, ax=ax[0], show=False, save=False, name=cluster_type, showname=False, verbose=False)
sc.pl.umap(adata[adata.obs.subtype!='other'], color='subtype',
           palette=monocyte_colormap, legend_fontsize=7, ax=ax[1], show=False)
fig.suptitle(f'2D_Projection_and_UMAP\n\n', fontsize=20)
plt.savefig(os.path.join(figure_path, f'subtype_immune_monocyte_projection_and_UMAP_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches = 'tight')
plt.close()

### CD8+

In [ ]:
tmp_colormap = dict()
for type_name in type_colormap.keys(): tmp_colormap[type_name]=(0.9,0.9,0.9)
tmp_colormap['CD8+'] = type_colormap['CD8+']

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
sc.pl.umap(adata[adata.obs.subtype!='other'], 
           color='type', palette=tmp_colormap, legend_fontsize=7, ax=ax, show=False)
plt.savefig(os.path.join(figure_path, 'type_nonimmune_CD8+_UMAP.png'), bbox_inches = 'tight')
plt.close()

In [ ]:
Subtype_of_interest = ['T_CD8+, GZMA+, CXCL13+', 'T_CD8+, CTLA4+', 'T_CD8+, PD1+', 'Cyto_T_CD8+']
CD8_colormap = dict()
for type_name in subtype_colormap.keys():
  CD8_colormap[type_name]=(0.9,0.9,0.9)  

CD8_colormap['T_CD8+, GZMA+, CXCL13+']=(1,0.8,0)
CD8_colormap['T_CD8+, CTLA4+']=(0.5,0.5,1)
CD8_colormap['T_CD8+, PD1+']=(1,0.4,0)
CD8_colormap['Cyto_T_CD8+']=(0.67,0.92,0.49)

plot_type = list(leiden_subtype_dict.keys())
for _ in Subtype_of_interest: plot_type.remove(_)
plot_type += Subtype_of_interest

linewidth = 1.5
dpi = 100
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
show_cluster(hulls, type_indices, cluster_list=plot_type, 
              cluster_colormap=CD8_colormap, linewidth=linewidth, ax=ax[0], show=False , save=False, name=cluster_type, showname=False, verbose=False)
sc.pl.umap(adata[adata.obs.subtype!='other'], color='subtype',
           palette=CD8_colormap, legend_fontsize=7, ax=ax[1], show=False)
fig.suptitle(f'2D_Projection_and_UMAP\n\n', fontsize=20)
plt.savefig(os.path.join(figure_path, f'subtype_immune_CD8_projection_and_UMAP_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches = 'tight')
plt.close()

### Neutrophil

In [ ]:
Subtype_of_interest = ['Neutrophil_CSF3R+, S100A8+', 'Neutrophil_CSF3R+']
Neu_colormap = dict()
for type_name in subtype_colormap.keys():
  Neu_colormap[type_name]=(0.9,0.9,0.9)  

Neu_colormap['Neutrophil_CSF3R+, S100A8+']=(1,0,1)
Neu_colormap['Neutrophil_CSF3R+']=(1,0.5,0)

plot_type = list(leiden_subtype_dict.keys())
for _ in Subtype_of_interest: plot_type.remove(_)
plot_type += Subtype_of_interest

linewidth = 1.5
dpi = 100
fig, ax = plt.subplots(ncols=2, nrows=1, figsize=(20, 10))
show_cluster(hulls, type_indices, cluster_list=plot_type, 
              cluster_colormap = Neu_colormap, linewidth=linewidth, ax=ax[0], show=False, save=False, name=cluster_type, showname=False, verbose=False)
sc.pl.umap(adata[adata.obs.subtype!='other'], color='subtype',
           palette=Neu_colormap, legend_fontsize=7, ax=ax[1], show=False)

fig.suptitle(f'2D_Projection_and_UMAP\n\n', fontsize=20)
plt.savefig(os.path.join(figure_path, f'subtype_immune_Neutrophil_projection_and_UMAP_linewidth={linewidth}_dpi={dpi}.png'), dpi=dpi, bbox_inches = 'tight')
plt.close()

### plot cDC subtypes

In [ ]:
cDC1_adata = adata[adata.obs['leiden'].isin(['34'])]
cDC2_adata = adata[adata.obs['leiden'].isin(['31', '88'])]

tmp_adata = adata[adata.obs.type!='other']
hulls, type_indices = create_hull(tmp_adata, clus_obs="leiden", cont_thre=5, 
                                rna_pos = pd.read_csv(os.path.join(base_path, 'segmented', "rna_labeled_0818_HCC.csv")))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
show_cluster(hulls, type_indices, cluster_list=['34'], 
             cluster_colormap={"34":"brown"}, 
            linewidth=2, ax=ax, show=False, save=False)
cell_num_ = len(tmp_adata)
fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
plt.tight_layout()
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'projection_cDC1_leiden_34.png'), dpi=300)
plt.close()

fig, ax = plt.subplots(figsize=(10, 10))
show_cluster(hulls, type_indices, cluster_list=['31','88'], 
             cluster_colormap={"31":"brown", "88":"brown"}, 
            linewidth=2, ax=ax, show=False, save=False)
cell_num_ = len(tmp_adata)
fig.suptitle(f'2D_Projection, cell_num={cell_num_}\n\n', fontsize=20)
plt.tight_layout()
plt.savefig(os.path.join(figure_path, 'projection_cDC2_leiden_31,88.png'), dpi=300)
plt.close()

### AFP+ Tumor colocalize with others

In [ ]:
from skimage.draw import polygon
from PIL import Image
from copy import deepcopy

downsize = 0.25
celltype_candidate_list = ['pDC_LILRA4+', 'T_CD4+, CXCL13+', 'B_MS4A1+', 'Plasma_B_CD79A+, MZB1+', 'Liver', 'other_cell_proliferation', 'Mast_CPA3+']
for celltype_candidate in celltype_candidate_list:
    subtype_colormap_tmp = deepcopy(subtype_colormap)
    for key in subtype_colormap_tmp.keys():
        if key not in ['Tumor_AFP+', celltype_candidate]: subtype_colormap_tmp[key] = (0.9,0.9,0.9)
        else: subtype_colormap_tmp[key] = subtype_colormap_tmp[key]

    # 参数设置
    pixel_resolution = 1  # 调整此值以提高分辨率（例如10.0）
    background_color = (255, 255, 255)  # 背景色（白色）

    adata_tmp = adata.copy()
    adata_tmp.obs['X_pos'] = adata_tmp.obsm['spatial'][:, 1] * downsize
    adata_tmp.obs['Y_pos'] = adata_tmp.obsm['spatial'][:, 0] * downsize

    # 获取坐标范围
    xmin, xmax = adata_tmp.obs['X_pos'].min(), adata_tmp.obs['X_pos'].max()
    ymin, ymax = adata_tmp.obs['Y_pos'].min(), adata_tmp.obs['Y_pos'].max()

    # 计算画布尺寸（增加边界避免坐标溢出）
    width_px = int((xmax - xmin) * pixel_resolution) + 200
    height_px = int((ymax - ymin) * pixel_resolution) + 200

    # 创建RGB画布
    canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)

    # 坐标转换函数（确保x,y顺序正确）
    def coord_to_px(x, y):
        px = int((x - xmin) * pixel_resolution) + 100
        py = int((y - ymin) * pixel_resolution) + 100
        return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)

    # 遍历细胞并绘制
    for idx, row in adata_tmp.obs.iterrows():
        idx = int(idx)
        try:
            if idx not in hulls:
                print(f"Warning: {idx} not in hulls, skipping.")
                continue
            hull = hulls[idx] * downsize
            celltype = row['subtype']
            color = subtype_colormap_tmp[celltype]
            color = tuple(int(255 * x) for x in color)
            if isinstance(color, str):
                if color.startswith('#'):
                    color = color.lstrip('#')
                    color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
                else:
                    # 处理命名颜色（如'red'需映射到RGB）
                    from matplotlib.colors import to_rgb
                    color = tuple(int(255 * x) for x in to_rgb(color))
            # 如果hull格式为[y, x]，需转换为[x, y]
            hull_xy = hull[:, [1, 0]]  # 假设hull原始格式为[y, x]x
            # 转换坐标到像素空间
            coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
            # 生成多边形像素坐标
            rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
            canvas[rr, cc, 0] = color[0]  # 红
            canvas[rr, cc, 1] = color[1]  # 绿
            canvas[rr, cc, 2] = color[2]  # 蓝
        except KeyError: continue

    imwrite(str(projection_path / f'tumor_afp_{celltype_candidate}.tiff'), np.flipud(canvas), photometric='rgb')
    # img = Image.fromarray(canvas)
    # img.save(str(projection_path / f"DC_tumor.png"), format='PNG')

# ROI

In [ ]:
ROI_path = projection_path / 'ROI'
ROI_path.mkdir(exist_ok=True)

In [ ]:
from skimage import io, transform, morphology

def ROI_mask_load(input_path, out_path='ROI_mask.png', show=True, save=False):
    ROI_mask = {}
    for mask_file in os.listdir(input_path):
        image = io.imread(os.path.join(input_path, mask_file))
        image = transform.rotate(image, angle=90, resize=True)
        image = morphology.binary_dilation(image, footprint=morphology.disk(5))
        ROI_mask[mask_file.replace('.tif','').replace('Mask', 'ROI')] = image
        
    ncols = int(-(-len(ROI_mask)**(1/2)//1))
    nrows = -(-len(ROI_mask)//ncols)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(ncols*4, nrows*4))
    for pos, mask_name in enumerate(list(ROI_mask.keys())):
        ax[pos // ncols][pos % ncols].imshow(ROI_mask[mask_name], cmap='gray')
        ax[pos // ncols][pos % ncols].set_title(mask_name)
        ax[pos // ncols][pos % ncols].set_xlabel("")
        ax[pos // ncols][pos % ncols].set_ylabel("")
    fig.suptitle('Mask_of_ROIs', fontsize=20)
    plt.tight_layout()

    if save: plt.savefig(out_path)
    elif show: plt.show()
    plt.close()

    return ROI_mask

In [ ]:
adata = sc.read_h5ad(os.path.join(typ_path, 'adata.h5ad'))
ROI_mask = ROI_mask_load(input_path=analysis_path/'roi_variation'/'roi_mask', show=False, save=False)
adata.obs['ROI'] = pd.Categorical(['other']*len(adata), categories=list(ROI_mask.keys()) + ['other'], ordered=False)    
adata.obs['Y_pos'] = adata.obsm['spatial'][:, 0]
adata.obs['X_pos'] = adata.obsm['spatial'][:, 1]
for _, mask in ROI_mask.items():
    yrange = mask.shape[0]
    for cell in tqdm(adata.obs.index, desc=_):
        if mask[yrange - int(adata.obs['Y_pos'].loc[cell]/100), int(adata.obs['X_pos'].loc[cell]/100)]:
            adata.obs['ROI'].loc[cell] = _
adata

In [ ]:
hulls = pickle.load(open(projection_path / 'hulls.pkl', 'rb'))
print(len(hulls))

## ROI overview

In [ ]:
from skimage.draw import polygon

# 参数设置
pixel_resolution = 1  # 调整此值以提高分辨率（例如10.0）
background_color = (255, 255, 255)  # 背景色（白色）

rois = [f'ROI_{i}' for i in range(1, 6)]
roi_colormap = {'ROI_1': (0.8, 0.2, 0.8), 'ROI_2': (0.2, 0.8, 0.2), 'ROI_3': (0.8, 0.6, 0.2), 'ROI_4': (0.8, 0.8, 0.2), 'ROI_5': (0.2, 0.8, 0.8), 'other': (0.8, 0.8, 0.8)}

In [ ]:

# 获取坐标范围
xmin, xmax = adata.obs['X_pos'].min(), adata.obs['X_pos'].max()
ymin, ymax = adata.obs['Y_pos'].min(), adata.obs['Y_pos'].max()

# 计算画布尺寸（增加边界避免坐标溢出）
width_px = int((xmax - xmin) * pixel_resolution) + 200
height_px = int((ymax - ymin) * pixel_resolution) + 200

# 创建RGB画布
canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)

# 坐标转换函数（确保x,y顺序正确）
def coord_to_px(x, y):
    px = int((x - xmin) * pixel_resolution) + 100
    py = int((y - ymin) * pixel_resolution) + 100
    return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)

# 遍历细胞并绘制
for idx, row in tqdm(adata.obs.iterrows()):
    idx = int(idx)
    try:
        # 关键修正1：检查hulls键是否匹配
        if idx not in hulls:
            print(f"Warning: {idx} not in hulls, skipping.")
            continue
        
        hull = hulls[idx]
        roi = row['ROI']
        
        # 关键修正2：处理颜色格式（支持十六进制和RGB元组）
        color = roi_colormap[roi]
        color = tuple(int(255 * x) for x in color)
        if isinstance(color, str):
            if color.startswith('#'):
                color = color.lstrip('#')
                color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
            else:
                # 处理命名颜色（如'red'需映射到RGB）
                from matplotlib.colors import to_rgb
                color = tuple(int(255 * x) for x in to_rgb(color))
        
        # 关键修正3：交换坐标顺序（根据hull实际存储结构）
        # 如果hull格式为[y, x]，需转换为[x, y]
        hull_xy = hull[:, [1, 0]]  # 假设hull原始格式为[y, x]
        
        # 转换坐标到像素空间
        coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
        
        # 生成多边形像素坐标
        rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
        
        # 关键修正4：正确填充RGB通道
        canvas[rr, cc, 0] = color[0]  # 红
        canvas[rr, cc, 1] = color[1]  # 绿
        canvas[rr, cc, 2] = color[2]  # 蓝
    except Exception as e:
        print(f"Error processing cell {idx}: {e}")
        continue

# downsample
canvas = canvas[::8, ::8]  # downsample by factor of 8

In [ ]:
import matplotlib.patches as mpatches
plt.imshow(np.flipud(canvas))
plt.legend(handles=[mpatches.Patch(color=roi_colormap[roi], label=roi) for roi in roi_colormap.keys()])
plt.axis('off')
plt.savefig(ROI_path / 'ROI_all.pdf')
plt.close()

In [ ]:
# 保存为TIFF（注意坐标系可能需要翻转）
imwrite(str(ROI_path / f'ROI_all.tiff'), np.flipud(canvas), photometric='rgb')

## cell type

In [ ]:
outpath = ROI_path / 'cell_type'
outpath.mkdir(exist_ok=True)

In [ ]:
import yaml
cell_typing_params = yaml.safe_load(open(os.path.join(analysis_path, 'cell_typing_params.yaml'), 'r'))
type_colormap = cell_typing_params['type_colormap']
subtype_colormap = dict()

leiden_annotation = cell_typing_params['leiden_annotation']
leiden_subtype_annotation = {}
for type_key, subtypes in leiden_annotation.items():
    type_values = []
    type_color = type_colormap[type_key]
    for subtype_key, values in subtypes.items():
        type_values.extend(values)
        leiden_subtype_annotation[subtype_key] = values
        subtype_colormap[subtype_key] = type_color
    leiden_annotation[type_key] = type_values

# subtype_colormap = {subtype:(1,1,1) for subtype in leiden_subtype_annotation.keys()}
# subtype_colormap = {subtype:(1,1,1) for subtype in leiden_subtype_annotation.keys()}
# subtype_colormap = {subtype:(0.9,0.9,0.9) for subtype in leiden_subtype_annotation.keys()}
subtype_colormap['Tumor_AFP+'] = (1,1,0)
subtype_colormap['Tumor_GPC3+'] = (1,0,0)
subtype_colormap['CAF_ACTA2+'] = (0,0,1)
subtype_colormap['Liver'] = (0.5,0.5,0.5)
# subtype_colormap['other'] = (1,1,1)
subtype_colormap['other'] = (0.9,0.9,0.9)

In [ ]:
from skimage.draw import polygon
from PIL import Image

# 参数设置
pixel_resolution = 1  # 调整此值以提高分辨率（例如10.0）
background_color = (255, 255, 255)  # 背景色（白色）

rois = [f'ROI_{i}' for i in range(1, 6)]

for roi in rois:
    adata_tmp = adata[adata.obs['ROI'] == roi]
    if adata_tmp.n_obs == 0: continue
    
    # 获取坐标范围
    xmin, xmax = adata_tmp.obs['X_pos'].min(), adata_tmp.obs['X_pos'].max()
    ymin, ymax = adata_tmp.obs['Y_pos'].min(), adata_tmp.obs['Y_pos'].max()
    
    # 计算画布尺寸（增加边界避免坐标溢出）
    width_px = int((xmax - xmin) * pixel_resolution) + 200
    height_px = int((ymax - ymin) * pixel_resolution) + 200
    
    # 创建RGB画布
    canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)
    
    # 坐标转换函数（确保x,y顺序正确）
    def coord_to_px(x, y):
        px = int((x - xmin) * pixel_resolution) + 100
        py = int((y - ymin) * pixel_resolution) + 100
        return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)
    
    # 遍历细胞并绘制
    for idx, row in adata_tmp.obs.iterrows():
        idx = int(idx)
        try:
            # 关键修正1：检查hulls键是否匹配
            if idx not in hulls:
                print(f"Warning: {idx} not in hulls, skipping.")
                continue
            
            hull = hulls[idx]
            celltype = row['subtype']
            
            # 关键修正2：处理颜色格式（支持十六进制和RGB元组）
            color = subtype_colormap[celltype]
            color = tuple(int(255 * x) for x in color)
            if isinstance(color, str):
                if color.startswith('#'):
                    color = color.lstrip('#')
                    color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
                else:
                    # 处理命名颜色（如'red'需映射到RGB）
                    from matplotlib.colors import to_rgb
                    color = tuple(int(255 * x) for x in to_rgb(color))
            
            # 关键修正3：交换坐标顺序（根据hull实际存储结构）
            # 如果hull格式为[y, x]，需转换为[x, y]
            hull_xy = hull[:, [1, 0]]  # 假设hull原始格式为[y, x]
            
            # 转换坐标到像素空间
            coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
            
            # 生成多边形像素坐标
            rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
            
            # 关键修正4：正确填充RGB通道
            canvas[rr, cc, 0] = color[0]  # 红
            canvas[rr, cc, 1] = color[1]  # 绿
            canvas[rr, cc, 2] = color[2]  # 蓝
        except KeyError: continue
    
    # 保存为TIFF（注意坐标系可能需要翻转）
    imwrite(str(outpath / f'{roi}.tiff'), np.flipud(canvas), photometric='rgb')
    img = Image.fromarray(canvas)
    img.save(str(outpath / f"{roi}.png"), format='PNG')

## gene expression

In [ ]:
from cmap import Colormap
from matplotlib.colors import ListedColormap


# Interpolate colors to create a smoother colormap
def interpolate_colors(colors, num_colors):
    """
    Interpolate a list of colors to create a smoother gradient.
    
    :param colors: List of color tuples.
    :param num_colors: Number of colors in the new colormap.
    :return: List of interpolated color tuples.
    """
    original_indices = np.linspace(0, 1, len(colors))
    new_indices = np.linspace(0, 1, num_colors)
    interpolated_colors = []

    for i in range(3):  # For R, G, B channels
        channel = np.array([c[i] for c in colors])
        interpolated_channel = np.interp(new_indices, original_indices, channel)
        interpolated_colors.append(interpolated_channel)

    return list(zip(interpolated_colors[0], interpolated_colors[1], interpolated_colors[2]))

def generate_colormap(cmap='imagej:fire', num_colors=256):
    # Load the 'fire' colormap using the custom Colormap class
    cmap_custom = Colormap(cmap)

    # Extract the colors from the custom colormap
    percentile = 100
    colors = [c for c in cmap_custom.iter_colors()]
    colors = [colors[_] for _ in range(len(colors) * percentile // 100)]

    # Interpolate to create a colormap with 256 colors
    smooth_colors = interpolate_colors(colors, num_colors)

    # Create a new ListedColormap
    cmap_custom = ListedColormap(smooth_colors)
    return cmap_custom

In [ ]:
thermal_LUT = pd.read_csv(analysis_path/'thermal_LUT.csv', index_col=0)
cmap = generate_colormap(thermal_LUT.values)
cmap

In [ ]:
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

def get_mapper(gene_exp,qmin=0.01,qmax=0.995, cmap=cmap):
    # 修改为分位数版本
    vmin = np.nanquantile(gene_exp, q=qmin)   # 低分位数（可自定义）
    vmax = np.nanquantile(gene_exp, q=qmax)  # 高分位数（可自定义）
    print(vmin, vmax)
    norm = Normalize(vmin=vmin, vmax=vmax)
    mapper = ScalarMappable(norm=norm, cmap=cmap)
    return mapper

In [ ]:
from skimage.draw import polygon

def gene_projection(idxs, gene_exp, X, Y, hulls, mapper, 
                    pixel_resolution=1.0, background_color=(255, 255, 255)):
    # 获取坐标范围
    xmin, xmax = X.min(), X.max()
    ymin, ymax = Y.min(), Y.max()
    
    # 计算画布尺寸
    width_px = int((xmax - xmin) * pixel_resolution) + 200
    height_px = int((ymax - ymin) * pixel_resolution) + 200
    
    # 创建画布
    canvas = np.full((height_px, width_px, 3), background_color, dtype=np.uint8)
    
    # 坐标转换函数
    def coord_to_px(x, y):
        px = int((x - xmin) * pixel_resolution) + 100
        py = height_px - 1 - (int((y - ymin) * pixel_resolution) + 100)
        return np.clip(px, 0, width_px-1), np.clip(py, 0, height_px-1)
    
    # 遍历细胞
    for i, idx in enumerate(idxs):
        try:
            # 获取对应gene表达值
            cell_exp = gene_exp[i]
            # 生成颜色（RGBA转RGB）
            color = tuple(np.array(mapper.to_rgba(cell_exp)[:3]) * 255)
            
            # 获取hull坐标
            hull = hulls[int(idx)]
            hull_xy = hull[:, [1, 0]]  # 调整坐标顺序
            
            # 转换坐标
            coords = np.array([coord_to_px(x, y) for x, y in hull_xy])
            rr, cc = polygon(coords[:, 1], coords[:, 0], (height_px, width_px))
            
            # 填充颜色
            canvas[rr, cc, 0] = int(color[0])
            canvas[rr, cc, 1] = int(color[1])
            canvas[rr, cc, 2] = int(color[2])

        except KeyError: continue
    return canvas

### HBV

In [ ]:
gene_name = 'HBV'
outpath = ROI_path / f'{gene_name}_exp'
outpath.mkdir(exist_ok=True)

gene_exp = np.array(adata.raw[:, gene_name].X.flatten())  # 转换为密集数组
mapper = get_mapper(gene_exp, qmin=0.01, qmax=0.995)

from PIL import Image
fig, axes = plt.subplots(ncols=len(rois), nrows=1, figsize=(5*len(rois), 5))
for i, roi in tqdm(enumerate(rois), total=len(rois)):
    adata_tmp = adata[adata.obs['ROI'] == roi]
    if adata_tmp.n_obs == 0: continue
    # save as TIFF
    canvas = gene_projection(idxs=adata_tmp.obs.index, hulls=hulls, gene_exp=adata_tmp.raw[:,gene_name].X.flatten(), X=adata_tmp.obs['X_pos'], Y=adata_tmp.obs['Y_pos'], mapper=mapper, )
    imwrite(str(outpath / f"{roi}_{gene_name}.tiff"), canvas, photometric='rgb')
    # Save as PNG
    img = Image.fromarray(canvas)
    img.save(str(outpath / f"{roi}_{gene_name}.png"), format='PNG')
    # plot
    axes[i].imshow(canvas)
    axes[i].set_title(roi)
    axes[i].axis('off')
plt.suptitle(f'{gene_name} expression in rois')
plt.show()

### AFP

In [ ]:
gene_name = 'AFP'
outpath = ROI_path / f'{gene_name}_exp'
outpath.mkdir(exist_ok=True)

gene_exp = np.array(adata.raw[:, gene_name].X.flatten())  # 转换为密集数组
mapper = get_mapper(gene_exp, qmin=0.01, qmax=0.995)

from PIL import Image
fig, axes = plt.subplots(ncols=len(rois), nrows=1, figsize=(5*len(rois), 5))
for i, roi in tqdm(enumerate(rois), total=len(rois)):
    adata_tmp = adata[adata.obs['ROI'] == roi]
    if adata_tmp.n_obs == 0: continue
    # save as TIFF
    canvas = gene_projection(idxs=adata_tmp.obs.index, hulls=hulls, gene_exp=adata_tmp.raw[:,gene_name].X.flatten(), X=adata_tmp.obs['X_pos'], Y=adata_tmp.obs['Y_pos'], mapper=mapper, )
    imwrite(str(outpath / f"{roi}_{gene_name}.tiff"), canvas, photometric='rgb')
    # Save as PNG
    img = Image.fromarray(canvas)
    img.save(str(outpath / f"{roi}_{gene_name}.png"), format='PNG')
    # plot
    axes[i].imshow(canvas)
    axes[i].set_title(roi)
    axes[i].axis('off')
plt.suptitle(f'{gene_name} expression in rois')
plt.show()

### GPC3

In [ ]:
gene_name = 'GPC3'
outpath = ROI_path / f'{gene_name}_exp'
outpath.mkdir(exist_ok=True)

gene_exp = np.array(adata.raw[:, gene_name].X.flatten())  # 转换为密集数组
mapper = get_mapper(gene_exp, qmin=0.01, qmax=0.995)

from PIL import Image
fig, axes = plt.subplots(ncols=len(rois), nrows=1, figsize=(5*len(rois), 5))
for i, roi in tqdm(enumerate(rois), total=len(rois)):
    adata_tmp = adata[adata.obs['ROI'] == roi]
    if adata_tmp.n_obs == 0: continue
    # save as TIFF
    canvas = gene_projection(idxs=adata_tmp.obs.index, hulls=hulls, gene_exp=adata_tmp.raw[:,gene_name].X.flatten(), X=adata_tmp.obs['X_pos'], Y=adata_tmp.obs['Y_pos'], mapper=mapper, )
    imwrite(str(outpath / f"{roi}_{gene_name}.tiff"), canvas, photometric='rgb')
    # Save as PNG
    img = Image.fromarray(canvas)
    img.save(str(outpath / f"{roi}_{gene_name}.png"), format='PNG')
    # plot
    axes[i].imshow(canvas)
    axes[i].set_title(roi)
    axes[i].axis('off')
plt.suptitle(f'{gene_name} expression in rois')
plt.show()

### MKI67

In [ ]:
gene_name = 'MKI67'
outpath = ROI_path / f'{gene_name}_exp'
outpath.mkdir(exist_ok=True)

gene_exp = np.array(adata.raw[:, gene_name].X.flatten())  # 转换为密集数组
mapper = get_mapper(gene_exp, qmin=0.01, qmax=0.995)

from PIL import Image
fig, axes = plt.subplots(ncols=len(rois), nrows=1, figsize=(5*len(rois), 5))
for i, roi in tqdm(enumerate(rois), total=len(rois)):
    adata_tmp = adata[adata.obs['ROI'] == roi]
    if adata_tmp.n_obs == 0: continue
    # save as TIFF
    canvas = gene_projection(idxs=adata_tmp.obs.index, hulls=hulls, gene_exp=adata_tmp.raw[:,gene_name].X.flatten(), X=adata_tmp.obs['X_pos'], Y=adata_tmp.obs['Y_pos'], mapper=mapper, )
    imwrite(str(outpath / f"{roi}_{gene_name}.tiff"), canvas, photometric='rgb')
    # Save as PNG
    img = Image.fromarray(canvas)
    img.save(str(outpath / f"{roi}_{gene_name}.png"), format='PNG')
    # plot
    axes[i].imshow(canvas)
    axes[i].set_title(roi)
    axes[i].axis('off')
plt.suptitle(f'{gene_name} expression in rois')
plt.show()